<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

# **Exercise 7:** Deploying Custom Applications on HPE Ezmeral Unified Analytics 

In the final exercise of this series, you will import and deploy a custom retail application on **HPE Ezmeral Unified Analytics**. 

This application will use your webcam to scan items as if you were at the checkout, detecting produce in the image using the model you created and served!

In this exercise, you will learn how to:

- Import applications and frameworks into Unified Analytics.
- Use the Smart Retail Experience application!

By the end of this exercise, you will have deployed an application that demonstrates what a next-generation retail experience could look like. 

Following this, you will have completed the Smart Retail Experience technical demonstration. 

Let's dive in!

<div class="alert alert-block alert-danger">
<b>Important:</b> This exercise requires the completion of Exercises 6: Serving your Model with KServe <b>and</b> a running Kubeflow Endpoint of your model!</div>

## **1. Importing Applications into HPE Ezmeral Unified Analytics**

### Importing the Smart Retail Experience application

1. Navigate back to the Unified Analytics dashboard.
1. In the sidebar navigation menu, select `Tools & Frameworks`.
1. Click `Import New Framework`.

<img src="./images/exercise7/frameworks.png" alt="Drawing" style="width: 50%;"/>

4. Under **Framework Details**, you will see the following:
- **Framework Name**: This will be the name of your application/framework that will appear in the `Tools and Frameworks` page. It is case sensitive. Default for this demonstraton: `Smart Retail Experience`.
- **Version**: The version of your application/framework that will appear in the `Tools and Frameworks` page. Best practice is to set it to the same version as the imported software. Default for this demonstration: `1.0.2`.
- **Description***: A general description of your application/framework that will appear as you write it here in the `Tools and Frameworks` page. Describe the function of the software (not 'MATLAB' as depicted above!). Write your own description for the Smart Retail Experience.
- **Category***: Applications/frameworks in HPE Ezmeral Unified Analytics fall under one of three categories: *Data Engineering*, *Analytics* and *Data Science*. Applications/frameworks under these categories are sorted into tabs in `Tools and Frameworks` page. For this demonstration, select `Data Science`.

5. Upload the `icon.png` located under the **retail_application** folder of this demonstration directory to the **Framework Icon** field.
6. Click `Framework Chart` to continue.

<img src="./images/exercise7/details.png" alt="Drawing" style="width: 40%;"/>

7. Under **Helm Chart** is where previously uploaded Helm charts stored by HPE Ezmeral Unified Analytics can be selected. As we are going to upload a new Helm Chart, leave the option as `Upload New Chart`
8. Upload the `retail-experience.tgz` located under the **retail_application** folder of this demonstration directory into the  **Upload Helm Package tar.gz file** field. The Framework Chart window will now look like this.

The **Namespace** field will dictate under what Kubernetes namespace your application will exist. The application **must** be in the same namespace as your Kubeflow Endpoints, which will exist in the namespace that is your Unified Analytics **username**. 

9. **Run the cell below to get the Namespace for your application deployment.**

In [ ]:
!cat /var/run/secrets/kubernetes.io/serviceaccount/namespace

10. Copy **the output of the cell above** into the **Namespace** field.
11. Click `Framework Values` to continue.

<img src="./images/exercise7/chart2.png" alt="Drawing" style="width: 40%;"/>

In the **Framework Values** page, you will find a preview of the Helm Values YAML (values.yaml). For a Helm chart to be uploaded to Unified Analytics, it must contain an `ezua` field which you can find below. This field will declare to the application where to find Unified Analytics security credentials for itself.

No changes are required to the YAML for this demonstration.

11. Click `Review` to continue.

<img src="./images/exercise7/values.png" alt="Drawing" style="width: 40%;"/>

12. In the **Review** window, click `Submit`.

With that, you have successfully uploaded **your own application to Unified Analytics!**

Navigate to the `Data Science` tab to view your application.

<img src="./images/exercise7/imported.png" alt="Drawing" style="width: 30%;"/>

**It will take several minutes to initialize even after the status changes to** `Ready`. **Please wait up to 5 minutes.**


## **2. Smart Retail Experience**

You are now ready to run the Smart Retail Experience application - leveraging 

To learn more about building Docker and Helm applications and creating your own Smart Retail Experience from scratch, keep an eye out on the **HPE Ezmeral GitHub repository** for the latest tutorials, demos and feature showcases!

### Remember EzPresto?

It now seems like so long ago, but recall the delta tables you turned into a Data Source in Unified Analytics using Apache Hive in [Exercises 1](./01.exploring_data_with_spark.ipynb) and [2](./02.query_with_ezpreso.ipynb). Instead of having to manually load prices into our self-serve checkout application, which will differ from store to store, would it not be best to **pull from a centeralized table to always have the latest prices**? 

Like in [Exercise 3](./03.visualizing_data_superset), where we pulled data into Apache Superset, we can once again leverage **EzPresto** to query our datasets that we create from our connected Data Sources - no matter where that data came from or happens to reside! 


The Smart Retail Experience application uses EzPresto to query the delta tables we connected through Hive. If you recall, we had **three** sales data tables - one each for our German, Swiss and Czech stores. 

Given the variance in pricing and currency, in order to test our Smart Retail Experience, you can **select which country you wish to get sales data for** from inside the Smart Retail Experience Application. 

Whenever an item is scanned, both the image frame and the selected `country` are parsed to the backend of the application. The frame will be sent to the model endpoint, where a prediction of what item is in the image will be returned. If a confidence threshold is met, that prediction (e.g. *orange*) is parsed as the `product` variable along with the `country` variable into an SQL query. This query is then made using EzPresto to our Cached Asset tables to get the last price of that item from that given country. 

```python

table_name = f'retail.default.{country}"

PRICE = f"SELECT unitprice FROM {table_name} WHERE PRODUCT = '{product}' LIMIT 1"
```

The prediction and retrieved price are then parsed back to the application, where the item is added to the cart!

**Note:** Given that no single in-store self-serve checkout will be needing to start a cart in two countries, when the country is changed in the application **the shopping cart will reset**. 


### Using the application

Before you get started, grab some items to scan! The only items that will be detected by the application are those that were trained on in [Exercise 4](./04.model_training.ipynb). 

For the `Quick` dataset, detected items will be:

- Apples
- Bananas
- Carrots
- Cucumbers
- Lemons
- Oranges

1. Click `Open` on your application. This will open the application in a new browser tab.
2. A browser pop-up should appear to allow for your access to your webcam.

Take a moment to play with the application. Most of the below steps you will explore on your own.

3. Place or hold an item from the list above in front of your webcam.
4. Click `Scan Item`. The image will now be processed.
5. The detected item will retrieve its price from the Presto database of the currently selected country. 
6. Switch the Scanning Mode to `Auto`.
7. When prompted by the status indicator, hold an item from the list above in front of your webcam.


### Detection Considerations

The detection threshold for any object is set to **80%**. That is, unless the model's predicted value for any given object is at least 80%, it will not add it to your cart.

With respect to the model's performance, please keep in mind that your model is **very lightweight** and was trained on a **small set of data**. It may make the occasional error. 

### Scanning

For demonstration purposes, there are two scanning modes in this applicaton: **Manual** and **Auto**

**Manual**: Click the "Scan Item" button with an item present in the image. The frame will be captured on button press and sent to the model for detection. Whilst you *can* scan another item whilst the status indicator reads *Processing...*, it is best to wait until the status indicator reads *Ready to scan item.* before scanning your next item. 

**Auto**: In Auto mode, the status indicator will indicate when to place your item. The status will change to *Scanning...* when there is two seconds left before capture. This image will then be processed. The timing indicator bar is used to visually indicate the cycle between the Scanning and Processing states - each 7 seconds. This is more analogous to a self-serve checkout solution, where a customer will continually scan items and as one item is being processed, they are getting the next item from their basket or trolley.

# **Conclusion**

Throughout the course of this technical demonstration, you have taken raw data from CSVs, created datasets you can easily query from anywhere, visualized those datasets in a C-suite ready dashboard, taken image data and created a product recognition model, served that model at scale to an endpoint, and deployed an application that leverages both the datasets and the served model to be used by thousands of customers... **all within a single platform!**

This technical demonstration has showcased the power of **HPE Ezmeral Unified Analytics** - the end-to-end data engieering, analytics, machine learning and AI platform that can be deployed on-premises or in your favorite public cloud vendor. 

Let's not forget, this demonstration has also showcased the power of... **you!** Leveraging Unified Analytics, you have performed every step of the machine learning development cycle - from data to model building. Be proud! 

From all of us at HPE, thank you for completing the **Smart Retail Experience** technical demonstration. We hope you had fun! 

Be sure to check out the other technical demonstrations and product tutorials in the **HPE Ezmeral GitHub Repository!**


Written and prepared by Alex Ollman, Product Manager, HPE Ezmeral Software.

Technical demonstration developed by Dirk Derichsweiler, Isabelle Steinhauser and Vincent Charbonnier. 


© Copyright 2024 Hewlett Packard Enterprise 